In [1]:
import requests
from datetime import datetime, timedelta
from bs4 import BeautifulSoup as bs
import json
from urllib.parse import urljoin

# 计算时间是否在一年内
def check_time(soup:bs):
    item_time = soup.find("time").get_text(separator=" ")

    time_temp_1 = item_time.split(" ")
    time_temp_2 = time_temp_1[1].split(".")

    day = int(time_temp_1[0])
    month = int(time_temp_2[1])
    year = int(time_temp_2[0])
    
    input_date = datetime(year=year, month=month, day=day)

    current_date = datetime.now()
    time_difference = abs(current_date - input_date)

    if time_difference <= timedelta(days=365):
        return (True, input_date.__str__())
    else:
        return (False, None)
    

def get_page(soup:bs):
    return int(soup.find_all("span", attrs={"class":"p_no"})[-1].get_text())


def get_li(soup:bs):
    return soup.find("ul", attrs={"class": "wdzxList"}).find_all('li')


class Item:
    def __init__(self, url, title, time):

        self.url = url
        self.title = title
        self.time = time


    def get_html(self):

        response = requests.get(self.url)
        response.encoding = 'utf-8'

        soup = bs(response.text, 'html.parser')
        self.content = soup.find("div", attrs={"class": "v_news_content"}).get_text()


    def store(self, path):

        if self.content is None:
            raise ValueError("出错")
        with open(path, 'w', encoding='utf-8') as f:
            f.write(json.dumps(self.__dict__, ensure_ascii=False, indent=4))

if __name__ == "__main__":
    name_li = ["wdyw", "zhxw", "hzjl", "yxcz"]
    error_li = []
    url_li = [f"https://news.whu.edu.cn/wdzx/{name}.htm" for name in name_li]
    count = 0

    for cur_url in url_li:
        page_count = 0
        is_continue = True
        
        while is_continue:
            response = requests.get(cur_url)
            response.encoding = 'utf-8'
            soup = bs(response.text, 'html.parser')
            page = get_page(soup)
            
            li = get_li(soup)
            for item in li:
                title = item.find("a").get("title")
                url = urljoin(cur_url, item.find("a").get("href"))
                time = check_time(item)
                if time[0]:
                    item = Item(url, title, time[1])
                    try:
                        item.get_html()
                        item.store(f"/data_share/zhangjiafeng/learn/lesson/data/{count}.json")
                        count += 1
                        print(url)
                    except:
                        error_li.append(url)
                else:
                    is_continue = False
                    break
            if is_continue:
                page_count += 1
                if page_count == 1:
                    cur_url = cur_url.replace(".htm", f"/{page - page_count}.htm")
                else:
                    cur_url = cur_url.replace(f"/{page - page_count + 1}.htm", f"/{page - page_count}.htm")
